In [27]:
import pandas as pd
from sqlalchemy.engine import URL
from sqlalchemy import create_engine

class ConnectionHander:
    def __init__(self, host, user, password, db):
        self.host = host
        self.user = user
        self.password = password
        self.db = db
        
        driver = "ODBC Driver 18 for SQL Server"
        
        connection_string = f'DRIVER={driver};SERVER={self.host};PORT=1433;DATABASE={self.db};UID={self.user};PWD={self.password};&autocommit=true;TrustServerCertificate=yes'
        connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
        self.engine = create_engine(connection_url, use_setinputsizes = False, echo = False)
        
        self.db_connection = self.engine.connect()
        
    def fetch_data(self, query):
        return pd.read_sql(query, con=self.db_connection)
    
CH = ConnectionHander('127.0.0.1,1500', 'SA', 'YBxUYjV7E0U', 'VoetbalDWH')



OperationalError: (pyodbc.OperationalError) ('08001', '[08001] [Microsoft][ODBC Driver 18 for SQL Server]TCP Provider: Timeout error [258].  (258) (SQLDriverConnect); [08001] [Microsoft][ODBC Driver 18 for SQL Server]Login timeout expired (0); [08001] [Microsoft][ODBC Driver 18 for SQL Server]Invalid connection string attribute (0); [08001] [Microsoft][ODBC Driver 18 for SQL Server]Unable to complete login process due to delay in prelogin response (258)')
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
## (4) In hoeveel procent van de wedstrijden wordt er niet gescoord?
query= 'select * from FactScore'
fact_score = CH.fetch_data(query)

# per wedstrijd de eindstand vinden
WedUistlag = fact_score.groupby('WedstrijdKey')['MinuutInWedstrijd'].max().reset_index()
# merge de eindstand met de fact_score tabel
EindScore = pd.merge(fact_score, WedUistlag, on=['WedstrijdKey', 'MinuutInWedstrijd'], how='inner')

# wedstrijden zonder doelpunt vinden
wedZonderDoelpunt = ((EindScore['ThuisScore'] == 0) & (EindScore['UitScore'] == 0)).sum()
percWedZonderDoelunt = (wedZonderDoelpunt / len(WedUistlag)) * 100


print(f"Aantal wedstrijden waarin niet wordt gescoord = {wedZonderDoelpunt}")
print(f"Totaal aantal wedstrijden = {len(WedUistlag)}")
print(f"Percentage = {percWedZonderDoelunt}")


Aantal wedstrijden waarin niet wordt gescoord = 4381
Totaal aantal wedstrijden = 18630
Percentage = 23.515834675254965


In [ ]:
## (5) Is er in de historische voetbaldata een ploeg terug te vinden die geen enkele of 1match gewonnen heeft gedurende de volledige competitie van één seizoen?

a='select * from dimDate'
b='select * from FactKlassement'
c='select * from dimClub'
Datum = CH.fetch_data(a)
Klassement = CH.fetch_data(b)
ploegen = CH.fetch_data(c)

ab=pd.merge(Klassement,Datum, left_on='EindeSpeeldagDateKey', right_on='DateKey', how='inner')
# eindstand per seizoen vinden
seizoenUitslag = ab.groupby('Seizoen')['AantalWedstrijden'].max().reset_index()
# eindstand van de competitie vinden met alle ander info erbij
al=pd.merge(ab,seizoenUitslag, on=['Seizoen', 'AantalWedstrijden'], how='inner')
# merge voor naam van ploeg te vinden
all=pd.merge(al,ploegen, left_on='ClubKey',right_on='ClubID', how='inner')
# kijken of het aantal gewonnen matchen 0 of 1 of 2 is
geenwin=all[all['AantalGewonnen']<1]
éénwin=all[all['AantalGewonnen']==1]
tweewin=all[all['AantalGewonnen']==2]
print(f"Ploegen die geen enkele match wonnen gedurende de volledige competitie (naam + seizoen):{geenwin[['Naam','Seizoen']]}")
print(f"Ploegen die 1 match wonnen gedurende de volledige competitie (naam + seizoen):{éénwin[['Naam','Seizoen']]}")
print(f"Ploegen die 2 matchen wonnen gedurende de volledige competitie (naam + seizoen):{tweewin[['Naam','Seizoen']]}")


Ploegen die geen enkele match wonnen gedurende de volledige competitie (naam + seizoen):Empty DataFrame
Columns: [Naam, Seizoen]
Index: []
Ploegen die 1 match wonnen gedurende de volledige competitie (naam + seizoen):Empty DataFrame
Columns: [Naam, Seizoen]
Index: []
Ploegen die 2 matchen wonnen gedurende de volledige competitie (naam + seizoen):            Naam Seizoen
335  KSC Hasselt   79/80
497     KRC Genk   88/89
731  KSK Beveren   01/02


In [ ]:
## (4) Wat is het langste aantal speeldagen dat een ploeg onklopbaar was (= gelijkgespeeld of gewonnen)? Welke was (waren) die ploeg(en)?

a='select * from dimDate'
b='select * from FactKlassement'
c='select * from dimClub'
dim_date = CH.fetch_data(a)
fact_klassement = CH.fetch_data(b)
dim_club = CH.fetch_data(c)
# merge om de correcte data te hebben
ab=pd.merge(fact_klassement,dim_date, left_on='EindeSpeeldagDateKey', right_on='DateKey', how='inner')
abc=pd.merge(dim_club,ab,left_on='ClubID', right_on='ClubKey', how='inner')
abc


,ClubID,Stamnummer,Naam,KlassementID,Rang,AantalWedstrijden,AantalGewonnen,AantalVerloren,AantalGelijk,DoelpuntenSaldo,...,EnglishMonthName,DutchMonthName,MonthOfQuarter,Quarter,QuarterName,Year,MonthYear,MMYYYY,Seizoen,Speeldag
0,1,216,Moeskroen,30745,11,1,0,1,0,-2,...,July,Juli,1,3,Third,2014,Jul-2014,072014,14/15,1.0
1,1,216,Moeskroen,30760,10,2,1,1,0,-1,...,August,Augustus,2,3,Third,2014,Aug-2014,082014,14/15,2.0
2,1,216,Moeskroen,30775,9,3,1,1,1,-1,...,August,Augustus,2,3,Third,2014,Aug-2014,082014,14/15,3.0
3,1,216,Moeskroen,30787,5,4,2,1,1,2,...,August,Augustus,2,3,Third,2014,Aug-2014,082014,14/15,4.0
4,1,216,Moeskroen,30803,5,5,2,1,2,2,...,August,Augustus,2,3,Third,2014,Aug-2014,082014,14/15,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35749,59,25,KV Mechelen,35827,9,25,9,10,6,-1,...,February,Februari,2,1,First,2024,Feb-2024,022024,23/24,25.0
35750,59,25,KV Mechelen,35842,8,26,10,10,6,0,...,February,Februari,2,1,First,2024,Feb-2024,022024,23/24,26.0
35751,59,25,KV Mechelen,35858,8,27,11,10,6,3,...,February,Februari,2,1,First,2024,Feb-2024,022024,23/24,27.0
35752,59,25,KV Mechelen,35874,8,28,12,10,6,4,...,March,Maart,3,1,First,2024,Mar-2024,032024,23/24,28.0


In [28]:
max = 0
PLoegOngeslaag = []
szn=[]
#Groeperen per naam voor later te loopen op naam
for _, groep in abc.groupby('Naam'):
    groep = groep.reset_index(drop=True)
    AantalOnslagsen = 0
    ploeg = None
    seizoen= None
    # loopen op per ploeg
    for i, row in groep.iterrows():
        # Als vorige lijn hetzelfde aantal verloren heeft als de huidige lijn dan +1
        if i > 0 and row['AantalVerloren'] == groep.loc[i - 1]['AantalVerloren']:
            AantalOnslagsen += 1
        else:
            AantalOnslagsen = 1 
            ploeg = row['Naam'] 
            seizoen= row['Seizoen']
        #indien er meer ongeslagen dagen zijn dan de vorige max ongeslagen wedstrijden
        if AantalOnslagsen > max:
            max = AantalOnslagsen
            PLoegOngeslaag = [ploeg]
            szn=[seizoen]
        #indien er meerdere ploegen zijn met hetzelfde aantal ongeslagen dagen zijn
        elif AantalOnslagsen == max:
            PLoegOngeslaag.append(ploeg)
            szn.append(seizoen)

max, PLoegOngeslaag,szn

(31, ['RSC Anderlecht', 'Standard Luik'], ['84/85', '07/08'])